In [ ]:
import os
from langchain_openai import ChatOpenAI
from rich.jupyter import print as pprint
from dotenv import load_dotenv
from langchain.tools import tool, BaseTool
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from typing import Optional, Literal, Annotated, List, Dict, Type
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from uuid import uuid4
import requests
import psycopg2
from langchain_community.tools.tavily_search import TavilySearchResults
import json
import yagmail

In [ ]:
load_dotenv()

#### Вспомогательные функции

In [ ]:
with open("devices.json", 'r', encoding='utf-8') as file:
    devices = json.load(file)

device_context = "\n".join(f"{d['friendly_name']}: {d['entity_id']}" for d in devices)

In [ ]:
def stream_graph_updates(graph, user_input, config=None):
    if user_input:
        data = {"messages": [HumanMessage(content=user_input)]}
    else:
        data = None
        
    for event in graph.stream(data, stream_mode="updates", config=config):
        node, data = list(event.items())[0]

        if "messages" in data and len(data['messages']) > 0:
            print("🤖 Петрушка: " + data["messages"][-1].content)
            #data["messages"][-1].pretty_print()

In [ ]:
base_model = ChatOpenAI(model='gpt-4o')
memory = MemorySaver()

In [ ]:
# Модели данных для инструментов

# Модели для управления умным домом
class HomeManageInput(BaseModel):
    device: str = Field(description="Уникальный ID устройства")
    action: Literal["turn_on", "turn_off"] = Field(description="Действие с устройством")

class HomeManageOutput(BaseModel):
    success: bool = Field(description="Успешность выполнения операции")
    message: str = Field(description="Сообщение о результате")
    device_id: Optional[str] = Field(default=None, description="ID устройства")

# Модели для поиска фильмов
class MovieSearchInput(BaseModel):
    genre: str = Field(description="Жанр для поиска")
    min_rating: float = Field(default=7.0, ge=0.0, le=10.0, description="Минимальный рейтинг IMDB")

class Movie(BaseModel):
    title: str = Field(description="Название фильма/сериала")
    type: str = Field(description="Тип (Фильм, Сериал, Мини-сериал)")
    year: int = Field(description="Год выпуска")
    genre: str = Field(description="Жанр")
    rating: float = Field(description="Рейтинг IMDB")

class MovieSearchOutput(BaseModel):
    movies: List[Movie] = Field(description="Список найденных фильмов и сериалов")

# Модели для создания задач
class TaskInput(BaseModel):
    task_title: str = Field(description="Название задачи")

class TaskOutput(BaseModel):
    success: bool = Field(description="Успешность создания задачи")
    message: str = Field(description="Сообщение о результате")
    task_title: Optional[str] = Field(default=None, description="Название созданной задачи")

In [ ]:
# Модель состояния
class State(BaseModel):
    messages: Annotated[list, add_messages]
    intent: Optional[str] = None

#### Определение намерения

In [ ]:
# Тип - что хочет пользователь
class IntentOutput(BaseModel):
    intent: Literal["control_light", "media_library", "task_management", "chat"] = Field(..., description="Определяет, что хочет пользователь")

In [ ]:
# NODE - Определяем функцию для извлечения намерения пользователя
def node_intent(state: State) -> dict:
    prompt = f"""
    Ты — полезный помощник. Определи, что хочет пользователь.

    Если запрос пользователя связан с управлением умным домом (темно, светло, включить или выключить), то это "intent": "control_light"

    Если запрос пользователя связан с его библиотекой фильмов (найти, посоветовать, добавить, посмотреть), то это "intent": "media_library"

    Если запрос пользователя связан с созданием задач (создай задачу, добавь в список дел, напомни, сделать что-то), то это "intent": "task_management"

    Если нет явного запроса на функции, то это просто общение и "intent": "chat"
    
    "{state.messages}"
    
    Ответь строго:
    {{"intent": "control_light"}} или {{"intent": "media_library"}} или {{"intent": "task_management"}} или {{"intent": "chat"}}
    """
    model = base_model.with_structured_output(IntentOutput)
    response = model.invoke(prompt)

    print("Намерение: " + response.intent)

    return {"intent" : response.intent}

#### Узел управления умным домом

In [ ]:
class HomeManageTool(BaseTool):
    name: str = "manage_home"
    description: str = "Включение и выключение устройств в умном доме"
    args_schema: Type[BaseModel] = HomeManageInput
    return_direct: bool = False

    def _run(self, device: str, action: str) -> HomeManageOutput:
        """Выполняет управление устройством умного дома"""
        
        url_map = {
            "turn_on": "http://homeassistant.local:8123/api/services/switch/turn_on",
            "turn_off": "http://homeassistant.local:8123/api/services/switch/turn_off",
        }

        if action not in url_map:
            return HomeManageOutput(
                success=False,
                message=f"Неподдерживаемое действие: {action}. Доступны: {list(url_map.keys())}",
                device_id=device
            )
        
        url = url_map[action]

        headers = {
            "Authorization": os.environ["HOME_ASSISTANT_TOKEN"],
            "Content-Type": "application/json"
        }
        
        data = {"entity_id": device}
        try:
            response = requests.post(url, headers=headers, json=data)
            if response.status_code == 200:
                return HomeManageOutput(
                    success=True,
                    message=f"Устройство {device} успешно {action}",
                    device_id=device
                )
            else:
                return HomeManageOutput(
                    success=False,
                    message=f"Ошибка при управлении устройством: {response.status_code}",
                    device_id=device
                )
        except Exception as e:
            return HomeManageOutput(
                success=False,
                message=f"Ошибка подключения к Home Assistant: {e}",
                device_id=device
            )

# Создаем экземпляр инструмента
manage_home = HomeManageTool()

In [ ]:
# NODE - управление умным домом
def node_home_manage(state: State) -> dict:
    
    custom_prompt=f"""
    Ты - помощник по умному дому, можешь включать и выключать устройства, если тебе это потребуется. Ты можешь оперировать вот таким списком соответствия названия устройства и его ID. В функцию включения нужно передавать именно ID:
    
    {device_context}
    
    """
    
    # Создаем агента
    tools = [manage_home]
    agent_executor = create_react_agent(base_model, tools, prompt=custom_prompt)
    
    # Вызываем агента с правильным форматом сообщений
    response = agent_executor.invoke({"messages": state.messages})
    
    # Возвращаем новые сообщения
    return {"messages": response["messages"]}
    

#### Узел работы с медиатекой

In [ ]:
class MovieSearchTool(BaseTool):
    name: str = "search_movies"
    description: str = "Поиск фильмов и сериалов по жанру и минимальному рейтингу"
    args_schema: Type[BaseModel] = MovieSearchInput
    return_direct: bool = False

    def _run(self, genre: str, min_rating: float = 7.0) -> MovieSearchOutput:
        """Выполняет поиск фильмов в базе данных"""
        
        try:
            # Установить соединение с базой данных
            connection = psycopg2.connect(
                user=os.getenv('DB_USER', ''),
                password=os.getenv('DB_PASSWORD', ''),
                host=os.getenv('DB_HOST', ''),
                port=os.getenv('DB_PORT', ''),
                database=os.getenv('DB_NAME', '')
            )
            
            cursor = connection.cursor()
            
            # SQL запрос с фильтрацией по жанру и рейтингу
            query = """
                SELECT title, type, year, genre, rating
                FROM movies
                WHERE genre ILIKE %s AND rating > %s
                ORDER BY rating DESC
                LIMIT 3;
            """

            # Выполнить запрос с параметрами
            genre_search = f'%{genre}%'
            cursor.execute(query, (genre_search, min_rating))
            results = cursor.fetchall()
            
            # Закрыть соединение
            cursor.close()
            connection.close()
            
            # Преобразовать результаты в список Movie объектов
            movies = []
            for row in results:
                movie = Movie(
                    title=row[0],
                    type=row[1],
                    year=int(row[2]),
                    genre=row[3],
                    rating=float(row[4])
                )
                movies.append(movie)
            
            return MovieSearchOutput(movies=movies)
            
        except Exception as e:
            print(f"Ошибка при подключении к базе данных: {e}")
            return MovieSearchOutput(movies=[])

# Создаем экземпляр инструмента
search_movies = MovieSearchTool()

In [ ]:
# NODE - работа с медиатекой
def node_media_library(state: State) -> dict:
    
    custom_prompt = """
    Ты - помощник по работе с медиатекой. Ты можешь искать фильмы и сериалы по жанрам и рейтингу.
    Помоги пользователю найти подходящий контент из его библиотеки.
    """

    # Создаем агента
    tools = [search_movies]
    agent_executor = create_react_agent(base_model, tools, prompt=custom_prompt)
    
    # Вызываем агента с правильным форматом сообщений
    response = agent_executor.invoke({"messages": state.messages})
    
    # Возвращаем новые сообщения
    return {"messages": response["messages"]}
    

#### Узел управления задачами

In [ ]:
class TaskTool(BaseTool):
    name: str = "create_task"
    description: str = "Создание задач в приложении Things"
    args_schema: Type[BaseModel] = TaskInput
    return_direct: bool = False

    def _run(self, task_title: str) -> TaskOutput:
        """Создает задачу через отправку email в Things"""
        
        try:
            yag = yagmail.SMTP(os.environ['EMAIL_LOGIN'], os.environ['EMAIL_PASSWORD'])
            
            yag.send(
                to=os.environ['THINGS_EMAIL'],
                subject=task_title,
                contents='Задача создана через голосового помощника'
            )
            
            return TaskOutput(
                success=True,
                message=f"Задача '{task_title}' успешно создана",
                task_title=task_title
            )
            
        except Exception as e:
            return TaskOutput(
                success=False,
                message=f"Ошибка при создании задачи: {e}",
                task_title=task_title
            )

# Создаем экземпляр инструмента
create_task = TaskTool()


In [ ]:
# NODE - управление задачами
def node_task_management(state: State) -> dict:
    
    custom_prompt = """
    Ты - помощник по управлению задачами. Ты можешь создавать задачи в приложении Things.
    Когда пользователь просит создать задачу, добавить в список дел или напомнить о чем-то, 
    используй инструмент create_task для создания соответствующей задачи.
    """

    # Создаем агента
    tools = [create_task]
    agent_executor = create_react_agent(base_model, tools, prompt=custom_prompt)
    
    # Вызываем агента с правильным форматом сообщений
    response = agent_executor.invoke({"messages": state.messages})
    
    # Возвращаем новые сообщения
    return {"messages": response["messages"]}


#### Узел общения

In [ ]:
# NODE - чат
def node_chat(state: State) -> dict:

    tool_search = TavilySearchResults(max_results=5)

    # Создаем агента
    tools = [tool_search]
    agent_executor = create_react_agent(base_model, tools)
    
    # Вызываем агента с правильным форматом сообщений
    response = agent_executor.invoke({"messages": state.messages})
    
    # Возвращаем новые сообщения
    return {"messages": response["messages"]}

#### Граф

In [ ]:
builder = StateGraph(State)

def route(state: State) -> str:
    if state.intent == "control_light":
        return "node_home_manage"
    elif state.intent == "media_library":
        return "node_media_library"
    elif state.intent == "task_management":
        return "node_task_management"
    elif state.intent == "chat":
        return "node_chat"
    return END

builder.add_node("node_intent", node_intent)
builder.add_node("node_home_manage", node_home_manage)
builder.add_node("node_media_library", node_media_library)
builder.add_node("node_task_management", node_task_management)
builder.add_node("node_chat", node_chat)


builder.add_conditional_edges("node_intent", route, {
    "node_home_manage": "node_home_manage",
    "node_media_library": "node_media_library",
    "node_task_management": "node_task_management",
    "node_chat": "node_chat",
    END: END
})
builder.add_edge(START, "node_intent")
builder.add_edge("node_home_manage", END)
builder.add_edge("node_media_library", END)
builder.add_edge("node_task_management", END)
builder.add_edge("node_chat", END)

graph = builder.compile(checkpointer=memory)

In [ ]:
graph

In [ ]:
config = {"configurable": {"thread_id": str(uuid4())}}

while True:
    user_input = input("👤 Кожаный:")
    
    if user_input.lower() in ["quit", "exit", "q"]:
        break
    
    stream_graph_updates(graph, user_input, config=config)

In [ ]:
#snapshot = graph.get_state(config)
#pprint(snapshot)

#for message in snapshot.values["messages"]:
#    message.pretty_print()

#state_history = list(graph.get_state_history(config))
#pprint(state_history)